In [ ]:
# import numpy as np
# import gurobipy as gp
# from gurobipy import GRB
# import networkx as nx

# # [parent, child]
# input_lst = [["X", "A"], 
#              ["l", "A"], 
#              ["l", "B"], 
#              ["Y", "B"]]
# hiddens_lst = ["l"]
# cards_dict = {"A": 2, "B": 2, "X": 2, "Y": 2, "l": 3}


# graph = nx.DiGraph()
# graph.add_edges_from((parent, child) for parent, child in input_lst)
    
# # set cardinalities
# cards_A, card_B, card_X, card_Y, card_l = 2,2,2,2,3


# def get_mvar_shape(var):
#     # returns tuple the product shape of var parents' cardinalities
#     parents = list(graph.predecessors(var))
#     return (var, *tuple(parents))


# for var in list(graph):
#     print(f'var {var}:', get_mvar_shape(var))

$$P(X_1, \cdots , X_n) = \prod_{i=1}^nP(X_i\vert Pa(X_i))$$

->
$$P(X_1, \cdots , X_k | P(X_{k+1}, \cdots , X_n)) =\sum_{\lambda_1, ..., \lambda_m} \prod_{\lambda_i \in \bm{\lambda}} P(\lambda_i | Pa(\lambda_i)) \prod_{X_i \in \bm{X}}P(X_i\vert Pa(X_i))$$

In [ ]:
# class BellCompatible:
#     def __init__(self, card_A, card_B, card_X, card_Y, card_l, verbose=0): # settings and hidden common cause cardinality
#         self.card_A = card_A
#         self.card_B = card_B
#         self.card_X = card_X
#         self.card_Y = card_Y
#         self.card_l = card_l
#         self.verbose = verbose
#         self.model = None

#     def initialize_model(self):
#         self.model = gp.Model("BellCompat")
#         self.model.reset()
#         self.model.setParam('OutputFlag', self.verbose)
#         self.model.params.NonConvex = 2 

#         # variables
#         self.P_l = self.model.addMVar(self.card_l, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="P_l")
#         self.P_A_giv_Xl = self.model.addMVar(shape=(self.card_A, self.card_X, self.card_l), vtype=GRB.CONTINUOUS, name="P(A|X,l)", lb=0, ub=1)
#         self.P_B_giv_Yl = self.model.addMVar(shape=(self.card_B, self.card_Y, self.card_l), vtype=GRB.CONTINUOUS, name="P(B|Y,l)", lb=0, ub=1)
#         self.prod = self.model.addMVar(shape=(self.card_B, self.card_l, self.card_Y), vtype=GRB.CONTINUOUS, name="P(B,l|Y)", lb=0, ub=1)
#         self.model.update()

#         # constraints
#         for b, y, l in np.ndindex(self.card_B, self.card_Y, self.card_l):
#             # bc can't multiply 3 vars in SEM constraint
#             self.model.addConstr(self.prod[b, l, y] == self.P_B_giv_Yl[b, y, l] * self.P_l[l], name=f"P(B|Y,l) * P(l)")

#         for x, l in np.ndindex(self.card_X, self.card_l):
#             self.model.addConstr(sum([self.P_A_giv_Xl[a, x, l] for a in range(self.card_A)]) == 1, name=f'sum P(a|{x,l}) = 1')
#         for y, l in np.ndindex(self.card_Y, self.card_l):
#             self.model.addConstr(sum([self.P_B_giv_Yl[b, y, l] for b in range(self.card_B)]) == 1, name=f'sum P(b|{y,l}) = 1')


#         self.model.addConstr(sum([self.P_l[l] for l in range(self.card_l)]) == 1, "sum_P_l = 1")

#     def is_compatible(self, P_AB_giv_XY): # P_AB_giv_XY array
#         self.initialize_model()

#         # structural equation
#         for a, b, x, y in np.ndindex(self.card_A, self.card_B, self.card_X, self.card_Y):
#             self.model.addConstr(P_AB_giv_XY[a, b, x, y] == sum([self.P_A_giv_Xl[a, x, l] * self.prod[b, l, y] for l in range(self.card_l)]), f"P(A,B|X,Y) = sum_l P(A|X,l)*P(B|Y,l)*P(l)")

#         self.model.update()
#         self.model.optimize()

#         if self.model.status == 2: # GRB.OPTIMAL
#             return True
#         else:
#             self.model.computeIIS()
#             return False
        

# ## testing:
# dist = np.random.rand(2,2,2,2)  # rand dist
# m = BellCompatible(2,2,2,2, card_l = 3, verbose=1)
# model_feasibility = m.is_compatible(dist)
# print("Is this compatible w/ Bell DAG?", model_feasibility)

In [148]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

class DAG_Dist_Compatibility:
    def __init__(self, graph, hiddens_lst, cards_dict): 
        self.graph = nx.DiGraph()
        self.graph.add_edges_from((parent, child) for parent, child in graph)
        self.verbose = 0
        self.hiddens_lst = hiddens_lst
        self.cards = cards_dict
        self.prob_vars = {}

    def get_mvar_details(self, var):
        parents = list(self.graph.predecessors(var))
        # returns tuple(card_var, *card_parents(var)), 
        return (self.cards[var], *tuple(self.cards[parent] for parent in parents)), "_".join(parents)

    def initialize_model(self):
        self.model = gp.Model("DAG-Dist Compatibility")
        self.model.reset()
        self.model.setParam('OutputFlag', self.verbose)
        self.model.params.NonConvex = 2

        ## variables
        for var in list(self.graph.nodes):
            shape, var_name = self.get_mvar_details(var)
            if not var_name:
                self.prob_vars[f'P_{var}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}')
            else:
                self.prob_vars[f'P_{var}_giv_{var_name}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}_giv_{var_name}')
        self.model.update()

    def markv_decomp_prod(self, coords, hidden_val):
        vars_to_multiply = []
        observable_indices = {var: idx for idx, var in enumerate(self.observables_lst)}

        for var in self.graph.nodes:
            if var in self.observables_lst:
                vars_to_multiply.append(var)


        # Reduce the list of variables to a single expression through auxiliary variables

        # return vars_to_multiply[0] if vars_to_multiply else 1  # Return the final product or 1 if empty
        return f"SEM: {vars_to_multiply}"



    def get_joint(self):
        self.observables_lst = [var for var in self.cards.keys() if var not in self.hiddens_lst]
        self.childless = [node for node in self.graph.nodes if self.graph.out_degree(node) == 0]
        self.given = [node for node in self.graph.nodes if node not in self.childless and node not in self.hiddens_lst]
        self.target_joint_prob = "".join(self.childless) + "_giv_" + "".join(self.given)
        self.observables_cards = tuple(self.cards[var] for var in self.observables_lst)
        self.hidden_cards = tuple(self.cards[var] for var in self.hiddens_lst)
        self.observable_combinations = itertools.product(*[range(card) for card in self.observables_cards])

        self.prob_vars[f'P_{self.target_joint_prob}'] = self.model.addMVar(shape= self.observables_cards, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{self.target_joint_prob}')
        self.model.update()
        
        # for a,b,x,y in np.ndindex(*tuple(self.observables_cards)): # <- generalize to any DAG by iterating through dictionary of observables
        # print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][0,0,0,0]} == {sum([self.markv_decomp_prod((0,0,0,0), l) for l in range(self.hidden_cards[0])])}")

        # specific debugging:
        # for a,b,z in np.ndindex(*tuple(self.observables_cards)):
        #     # print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][a,b,z]} == {sum([self.markv_decomp_prod((a,b,z), l) for l in range(self.hidden_cards[0])])}")
        #     print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][a,b,z]} == {[self.markv_decomp_prod((a,b,z), l) for l in range(self.hidden_cards[0])]}") 
        #     break
        # print(self.markv_decomp_prod((0,0,0), 2))


        # LHS would iterate through self.target_joint_pro, i.e. self.prob_vars[f'P_{self.target_joint_prob}']
        cards_tmp = [self.cards[var] for var in self.observables_lst]

        ## variables
        sem_lst = []
        sem_var_count = 0
        print(self.prob_vars.keys())
        for var in list(self.graph.nodes):
            sem_var_count += 1
            _, var_name = self.get_mvar_details(var)
            if not var_name:
                sem_lst.append(f'P_{var}')
            else:
                sem_lst.append(f'P_{var}_giv_{var_name}')
        # print(    , sem_lst)

        # make the sem_var_count variables less so that can multiply them
        # iterate through cards_tmp

        # for a,b,z in np.ndindex(*tuple(self.observables_cards)):
        for comb in itertools.product(*[range(card) for card in cards_tmp]):

            # print(f"P_{self.target_joint_prob} == \n{self.prob_vars[f'P_A_giv_Z_l'][0,0,0]} * {self.prob_vars[f'P_B_giv_l'][0,0]} * {self.prob_vars[f'P_Z'][0]}")
            # print(comb, f"""{self.prob_vars[f'P_{self.target_joint_prob}'][comb]} 
            #       == {self.prob_vars[f'P_A_giv_Z_l'][comb[0],comb[2],0]}
            #       * {self.prob_vars[f'P_B_giv_l'][comb[1],0]} 
            #       * {self.prob_vars[f'P_Z'][0]} 
            #       * {self.prob_vars[f'P_l'][0]}""")
            # # break




            # print(comb, f"""{self.prob_vars[f'P_{self.target_joint_prob}'][comb]} 
            #       == [{[
            #         self.prob_vars[f'P_A_giv_Z_l'][comb[0],comb[2],hidden1]
            #         *
            #         self.prob_vars[f'P_B_giv_l'][comb[1],hidden1]
            #         *
            #         self.prob_vars[f'P_Z'][hidden1]
                                              
            #            for hidden1 in range(self.hidden_cards[0])]}]
            #       """)
                  
                #   {self.prob_vars[f'P_A_giv_Z_l'][comb[0],comb[2],0]}
                #   * {self.prob_vars[f'P_B_giv_l'][comb[1],0]} 
                #   * {self.prob_vars[f'P_Z'][0]} 
                #   * {self.prob_vars[f'P_l'][0]}""")

            break


        # print(*tuple(self.observables_cards))


# [parent, child]

# input_lst = [["X", "A"], ["l", "A"], ["l", "B"], ["Y", "B"]]
# cards_dict = {"A": 2, "B": 2, "X": 2, "Y": 2, "l": 3}
input_lst = [["Z", "A"], ["l", "A"], ["l", "B"]]
cards_dict = {"A": 2, "B": 2, "Z": 2, "l": 3}
hiddens_lst = ["l"]


example = DAG_Dist_Compatibility(input_lst, hiddens_lst, cards_dict)
example.initialize_model()
example.get_joint()

Discarded solution information
dict_keys(['P_Z', 'P_A_giv_Z_l', 'P_l', 'P_B_giv_l', 'P_AB_giv_Z'])


In [10]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

class DAG_Dist_Compatibility:
    def __init__(self, graph, hiddens_lst, cards_dict): 
        self.graph = nx.DiGraph()
        self.graph.add_edges_from((parent, child) for parent, child in graph)
        self.verbose = 0
        self.hiddens_lst = hiddens_lst
        self.cards = cards_dict
        self.prob_vars = {}

    def get_mvar_details(self, var):
        parents = list(self.graph.predecessors(var))
        # returns tuple(card_var, *card_parents(var)), 
        return (self.cards[var], *tuple(self.cards[parent] for parent in parents)), "_".join(parents)

    def initialize_model(self):
        self.model = gp.Model("DAG-Dist Compatibility")
        self.model.reset()
        self.model.setParam('OutputFlag', self.verbose)
        self.model.params.NonConvex = 2

        ## variables
        for var in list(self.graph.nodes):
            shape, var_name = self.get_mvar_details(var)
            if not var_name:
                self.prob_vars[f'P_{var}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}')
            else:
                self.prob_vars[f'P_{var}_giv_{var_name}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}_giv_{var_name}')
        self.model.update()

    """
    def markv_decomp_prod(self, coords, hidden_val):
        vars_to_multiply = []
        for var in list(self.graph.nodes):
            _, name = self.get_mvar_details(var)
            if not name:
                vars_to_multiply.append(self.prob_vars[f'P_{var}'][0])
            else:
                vars_to_multiply.append(self.prob_vars[f'P_{var}_giv_{name}'][0, 0, hidden_val])  # example index

        while len(vars_to_multiply) > 1:
            new_vars_to_multiply = []
            for i in range(0, len(vars_to_multiply), 2):
                if i+1 < len(vars_to_multiply):
                    # create an auxiliary variable for each pair of variables
                    aux_var_name = f'aux_{i//2}'
                    self.prob_vars[aux_var_name] = self.model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=aux_var_name)
                    self.model.addConstr(self.prob_vars[aux_var_name] == vars_to_multiply[i] * vars_to_multiply[i+1])
                    new_vars_to_multiply.append(self.prob_vars[aux_var_name])
                else:
                    new_vars_to_multiply.append(vars_to_multiply[i])
            vars_to_multiply = new_vars_to_multiply

        return vars_to_multiply[0] if vars_to_multiply else 1  # Return the final product or 1 if empty
    """

    def markv_decomp_prod(self, coords, hidden_val):
        vars_to_multiply = []
        observable_indices = {var: idx for idx, var in enumerate(self.observables_lst)}

        for var in self.graph.nodes:
            card, *parents = self.get_mvar_details(var)
            # If no parents, access the unconditioned probability variable
            if not parents:
                vars_to_multiply.append(self.prob_vars[f'P_{var}'][0])  # assuming 1 dim distribution
            else:
                # Construct the index tuple from coords based on the parent names
                indices = []
                for parent in parents:
                    if parent in observable_indices:  # Parent is an observable
                        indices.append(coords[observable_indices[parent]])
                    elif parent == self.hiddens_lst[0]:  # Assuming single hidden variable for simplicity
                        indices.append(hidden_val)

                # Ensure we use the right key for conditioned probabilities
                parent_string = '_'.join(parents)
                var_key = f'P_{var}_giv_{parent_string}' if parent_string else f'P_{var}'

                print(f"var_key: {var_key}, indices: {tuple(indices)}")
                vars_to_multiply.append(self.prob_vars[var_key][tuple(indices)])

        # Reduce the list of variables to a single expression through auxiliary variables
        while len(vars_to_multiply) > 1:
            new_vars_to_multiply = []
            for i in range(0, len(vars_to_multiply), 2):
                if i + 1 < len(vars_to_multiply):
                    aux_var_name = f'aux_{i // 2}'
                    self.prob_vars[aux_var_name] = self.model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=aux_var_name)
                    self.model.addConstr(self.prob_vars[aux_var_name] == vars_to_multiply[i] * vars_to_multiply[i + 1])
                    new_vars_to_multiply.append(self.prob_vars[aux_var_name])
                else:
                    new_vars_to_multiply.append(vars_to_multiply[i])
            vars_to_multiply = new_vars_to_multiply

        return vars_to_multiply[0] if vars_to_multiply else 1  # Return the final product or 1 if empty



    def get_joint(self):
        self.observables_lst = [var for var in self.cards.keys() if var not in self.hiddens_lst]
        self.childless = [node for node in self.graph.nodes if self.graph.out_degree(node) == 0]
        self.given = [node for node in self.graph.nodes if node not in self.childless and node not in self.hiddens_lst]
        self.target_joint_prob = "".join(self.childless) + "_giv_" + "".join(self.given)
        self.observables_cards = tuple(self.cards[var] for var in self.observables_lst)
        self.hidden_cards = tuple(self.cards[var] for var in self.hiddens_lst)
        self.observable_combinations = itertools.product(*[range(card) for card in self.observables_cards])

        self.prob_vars[f'P_{self.target_joint_prob}'] = self.model.addMVar(shape= self.observables_cards, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{self.target_joint_prob}')
        self.model.update()
        
        # for a,b,x,y in np.ndindex(*tuple(self.observables_cards)): # <- generalize to any DAG by iterating through dictionary of observables
        #     print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][0,0,0,0]} == {sum([self.markv_decomp_prod((0,0,0,0), l) for l in range(self.hidden_cards[0])])}")

        # specific debugging:
        for a,b,z in np.ndindex(*tuple(self.observables_cards)):
            # print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][a,b,z]} == {sum([self.markv_decomp_prod((a,b,z), l) for l in range(self.hidden_cards[0])])}")
            print(f"{self.prob_vars[f'P_{self.target_joint_prob}'][a,b,z]} == {[self.markv_decomp_prod((a,b,z), l) for l in range(self.hidden_cards[0])]}") 
            break
        # print(self.markv_decomp_prod((0,0,0), 2))
        # print(self.prob_vars.keys())
        # print(self.prob_vars)


# [parent, child]

# input_lst = [["X", "A"], ["l", "A"], ["l", "B"], ["Y", "B"]]
# cards_dict = {"A": 2, "B": 2, "X": 2, "Y": 2, "l": 3}
input_lst = [["Z", "A"], ["l", "A"], ["l", "B"]]
cards_dict = {"A": 2, "B": 2, "Z": 2, "l": 3}
hiddens_lst = ["l"]


example = DAG_Dist_Compatibility(input_lst, hiddens_lst, cards_dict)
example.initialize_model()
example.get_joint()

Discarded solution information
var_key: P_Z, indices: ()
var_key: P_A_giv_Z_l, indices: ()
var_key: P_l, indices: ()
var_key: P_B_giv_l, indices: (0,)


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (2,) and arg 1 with shape (2, 2, 3).

: 